In [50]:
import pandas as pd
import numpy as np
import io
import gc
import time
import pickle
from pprint import pprint
from datetime import date
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# settings
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [12]:
!pip3 install imbalanced-learn > /dev/null!pip3 install lightgbm > /dev/null
!pip3 install catboost > /dev/null
!pip3 install xgboost > /dev/null

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Global Variables
random_state = 50

In [15]:
ROOT_PATH = 'C:\\Users\\Usuario\\Desktop\\Proyecto Final\\src\\data\\'
ONLINE_PATH ='/content/drive/MyDrive/'

In [16]:
df_train = pd.read_csv(ONLINE_PATH+'train_preprocessed.csv')
df_test = pd.read_csv(ONLINE_PATH+'test_preprocessed.csv')

In [17]:
df_train.shape

(18047, 32)

In [18]:
df_test.shape

(9465, 30)

In [19]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass
9511,6.0,0.0,0.0,1.0,1.0,4.995870,48.0,64.0,1.0,2.0,1.0,2.0,3.0,1.0,4.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,2.0,9.712615,2.0,1.0,0.0,0.0,0.0,1.0,Mitochondrial genetic inheritance disorders,Leigh syndrome
13496,13.0,1.0,1.0,1.0,0.0,5.011423,27.0,20.0,0.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,1.0,1.0,0.0,2.0,4.841258,1.0,0.0,0.0,1.0,0.0,0.0,Single-gene inheritance diseases,Tay-Sachs
10574,8.0,1.0,1.0,1.0,0.0,4.572720,30.0,46.0,0.0,1.0,2.0,1.0,2.0,4.0,3.0,2.0,1.0,3.0,3.0,1.0,1.0,4.0,2.0,12.000000,2.0,0.0,1.0,1.0,1.0,1.0,Multifactorial genetic inheritance disorders,Diabetes


In [20]:
df_train['GeneticDisorder-DisorderSubclass'] = df_train['Genetic Disorder'] + '<->' + df_train['Disorder Subclass']

In [21]:
df_train.sample(3)

,Patient Age,Genes in mother's side,Inherited from father,Maternal gene,Paternal gene,Blood cell count (mcL),Mother's age,Father's age,Status,Respiratory Rate (breaths/min),Heart Rate (rates/min,Follow-up,Gender,Birth asphyxia,Autopsy shows birth defect (if applicable),Folic acid details (peri-conceptional),H/O serious maternal illness,H/O radiation exposure (x-ray),H/O substance abuse,Assisted conception IVF/ART,History of anomalies in previous pregnancies,No. of previous abortion,Birth defects,White Blood cell count (thousand per microliter),Blood test result,Symptom 1,Symptom 2,Symptom 3,Symptom 4,Symptom 5,Genetic Disorder,Disorder Subclass,GeneticDisorder-DisorderSubclass
89,0.0,1.0,0.0,0.000000,0.0,5.010995,30.0,38.0,1.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,2.000000,0.000000,1.0,1.495135,2.08256,2.0,12.000000,2.0,1.00000,1.0,1.0,0.0,0.0,Single-gene inheritance diseases,Tay-Sachs,Single-gene inheritance diseases<->Tay-Sachs
9345,11.0,1.0,1.0,0.581772,0.0,5.411270,41.0,56.0,0.0,1.0,2.0,1.0,3.0,1.0,3.0,1.0,2.0,2.239298,2.224563,2.0,1.524919,0.00000,2.0,5.116814,2.0,0.60561,1.0,0.0,1.0,0.0,Mitochondrial genetic inheritance disorders,Leigh syndrome,Mitochondrial genetic inheritance disorders<->...
9543,14.0,0.0,1.0,0.000000,1.0,4.762194,30.0,31.0,0.0,1.0,1.0,1.0,3.0,1.0,3.0,1.0,1.0,0.000000,2.000000,2.0,2.000000,3.00000,1.0,3.677022,4.0,0.00000,0.0,1.0,0.0,1.0,Mitochondrial genetic inheritance disorders,Leigh syndrome,Mitochondrial genetic inheritance disorders<->...


### Checando si el dataset esta balanceado

In [22]:
target_count = df_train['GeneticDisorder-DisorderSubclass'].value_counts()
target_count

Mitochondrial genetic inheritance disorders<->Leigh syndrome                         4683
Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy                 3971
Single-gene inheritance diseases<->Cystic fibrosis                                   3145
Single-gene inheritance diseases<->Tay-Sachs                                         2556
Multifactorial genetic inheritance disorders<->Diabetes                              1653
Single-gene inheritance diseases<->Hemochromatosis                                   1228
Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy     587
Multifactorial genetic inheritance disorders<->Alzheimer's                            133
Multifactorial genetic inheritance disorders<->Cancer                                  91
Name: GeneticDisorder-DisorderSubclass, dtype: int64

### Separando los datos

In [23]:
target_labels = df_train['GeneticDisorder-DisorderSubclass'].values

df_train.drop(['Genetic Disorder','Disorder Subclass', 'GeneticDisorder-DisorderSubclass'], axis=1, inplace=True)

In [24]:
# Split for genetic_disorder_labels
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_train, target_labels, test_size=0.1, random_state=random_state)

### Over Sampling usando SMOTE(Synthetic Minority Oversampling Technique)

In [25]:
# https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/
from imblearn.over_sampling import SMOTE
smote_overSampling = SMOTE()
X_train,y_train = smote_overSampling.fit_resample(X_train,y_train)
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{"Mitochondrial genetic inheritance disorders<->Leber's hereditary optic neuropathy": 4192,
 'Mitochondrial genetic inheritance disorders<->Leigh syndrome': 4192,
 'Mitochondrial genetic inheritance disorders<->Mitochondrial myopathy': 4192,
 "Multifactorial genetic inheritance disorders<->Alzheimer's": 4192,
 'Multifactorial genetic inheritance disorders<->Cancer': 4192,
 'Multifactorial genetic inheritance disorders<->Diabetes': 4192,
 'Single-gene inheritance diseases<->Cystic fibrosis': 4192,
 'Single-gene inheritance diseases<->Hemochromatosis': 4192,
 'Single-gene inheritance diseases<->Tay-Sachs': 4192}

### Escalando los datos: genetic_disorder

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)

X_test_scaled = scaler.transform(df_test)

X_train_scaled

array([[ 1.25874215, -1.34896194, -0.90318645, ..., -1.24784251,
         1.02583477, -1.10204476],
       [ 0.01385968, -1.34896194, -0.90318645, ..., -1.24784251,
         1.02583477,  1.0703738 ],
       [ 1.75669514,  0.86939053,  1.3252865 , ..., -1.24784251,
         1.02583477,  1.0703738 ],
       ...,
       [-1.47999929,  0.86939053, -0.80484041, ...,  0.93096645,
        -1.11317177, -1.05707069],
       [ 0.06974199,  0.30615737,  0.75948378, ..., -0.36980665,
        -1.1628251 , -0.55047408],
       [ 1.16786555, -0.53926165,  1.3252865 , ..., -1.24784251,
        -0.36396263, -1.10204476]])

# Elegiendo modelos y ajustando hiperparametros

### **GradientBoostingClassifier**

In [ ]:
%%time

from sklearn.ensemble import GradientBoostingClassifier

gb_clas = GradientBoostingClassifier(random_state=random_state)

# GRADIENT BOOSTING
grid_gradient_boosting_param = {"loss": ["deviance"],
                          "learning_rate": [0.1, 0.2],
                          "n_estimators": [100,200], 
                          "max_depth": [4,5], 
                          "max_features": ["sqrt", 3, 4],
                          }
Grid_gbct = RandomizedSearchCV(estimator=gb_clas, param_distributions=grid_gradient_boosting_param, verbose=1, n_iter=200, scoring='f1_weighted', cv=5)
Grid_gbct.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Wall time: 2h 22min 31s


RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=50),
                   n_iter=200,
                   param_distributions={'learning_rate': [0.1, 0.2],
                                        'loss': ['deviance'],
                                        'max_depth': [4, 5],
                                        'max_features': ['sqrt', 3, 4],
                                        'n_estimators': [100, 200]},
                   scoring='f1_weighted', verbose=1)

### **XGBoostClassifier**

In [27]:
%%time
import xgboost


#xgb_clas = xgboost.XGBClassifier(objective='multi:softmax',metric='multiclass',
#                                  eval_metric='mlogloss', num_iterations=1000,random_state=random_state)


xgb_clas = xgboost.XGBClassifier(eval_metric='mlogloss',verbose = 1,random_state=random_state)


# A parameter grid for XGBoost
grid_xgb_params = {
        "learning_rate": [0.1],
        'gamma': [ 1, 2],
        "n_estimators": [600],
        'max_depth': [4, 5]
        }


Grid_xgb = RandomizedSearchCV(estimator=xgb_clas, param_distributions=grid_xgb_params, verbose=1, n_iter=100, scoring='f1_weighted', cv=3)
Grid_xgb.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 56.9min finished


CPU times: user 1h 5min 9s, sys: 10.3 s, total: 1h 5min 19s
Wall time: 1h 5min 2s


### **LGBMClassifier**

In [ ]:
%%time
from lightgbm import LGBMClassifier

lgbm_clas = LGBMClassifier(objective='multi:softmax',random_state=random_state)

# A parameter grid for XGBoost

grid_lgbm_params = {
    'objective': 'multiclass',
    "n_estimators": [600,700],
    'learning_rate': [0.1],
    'max_depth': [6, 7]
}


Grid_lgbm = RandomizedSearchCV(estimator=lgbm_clas, param_distributions=grid_lgbm_params, verbose=1, n_iter=200, scoring='f1_weighted', cv=5)
Grid_lgbm.fit(X_train_scaled, y_train, eval_set=(X_cv_scaled, y_cv))

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Se han truncado las últimas 5000 líneas del flujo de salida.
[503]	valid_0's multi_logloss: 1.50573
[504]	valid_0's multi_logloss: 1.50629
[505]	valid_0's multi_logloss: 1.50669
[506]	valid_0's multi_logloss: 1.50721
[507]	valid_0's multi_logloss: 1.50767
[508]	valid_0's multi_logloss: 1.50842
[509]	valid_0's multi_logloss: 1.50924
[510]	valid_0's multi_logloss: 1.5097
[511]	valid_0's multi_logloss: 1.50959
[512]	valid_0's multi_logloss: 1.51006
[513]	valid_0's multi_logloss: 1.51049
[514]	valid_0's multi_logloss: 1.51142
[515]	valid_0's multi_logloss: 1.51171
[516]	valid_0's multi_logloss: 1.51207
[517]	valid_0's multi_logloss: 1.5127
[518]	valid_0's multi_logloss: 1.51277
[519]	valid_0's multi_logloss: 1.51326
[520]	valid_0's multi_logloss: 1.51417
[521]	valid_0's multi_logloss: 1.51461
[522]	valid_0's multi_logloss: 1.51447
[523]	valid_0's multi_logloss: 1.51508
[524]	valid_0's multi_logloss: 1.5163
[525]	valid_0's multi_logloss: 1.51708
[526]	valid_0's multi_logloss: 1.518
[527]	va

### **CatboostClassifier** 

In [39]:
%%time

from catboost import CatBoostClassifier

cbc_clas = CatBoostClassifier(loss_function='MultiClass', use_best_model=True, task_type="GPU")

# Best Params: {'random_strength': 0.5, 'max_ctr_complexity': 3, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 10, 'border_count': 200, 'bagging_temperature': 0.03}

grid_cbc_params = {
    'depth':[9,10],
    'iterations':[100,500],
    'learning_rate':[0.1],
    'l2_leaf_reg':[1,5],
    'border_count':[100,200],
    'bagging_temperature':[0.03,0.09],
    'random_strength':[0.5],
    'max_ctr_complexity':[3]
}

Grid_cbc = RandomizedSearchCV(estimator=cbc_clas, param_distributions=grid_cbc_params, verbose=1, n_iter=200, scoring='f1_weighted', cv=5)
Grid_cbc.fit(X_train_scaled, y_train, eval_set=(X_cv_scaled, y_cv))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Se han truncado las últimas 5000 líneas del flujo de salida.
30:	learn: 0.7868447	test: 1.3861492	best: 1.3861492 (30)	total: 3.19s	remaining: 48.3s
31:	learn: 0.7737864	test: 1.3823382	best: 1.3823382 (31)	total: 3.3s	remaining: 48.2s
32:	learn: 0.7613014	test: 1.3783631	best: 1.3783631 (32)	total: 3.4s	remaining: 48.2s
33:	learn: 0.7490140	test: 1.3752183	best: 1.3752183 (33)	total: 3.5s	remaining: 47.9s
34:	learn: 0.7359639	test: 1.3726695	best: 1.3726695 (34)	total: 3.6s	remaining: 47.9s
35:	learn: 0.7250556	test: 1.3692111	best: 1.3692111 (35)	total: 3.71s	remaining: 47.8s
36:	learn: 0.7141334	test: 1.3652982	best: 1.3652982 (36)	total: 3.8s	remaining: 47.6s
37:	learn: 0.7045365	test: 1.3632734	best: 1.3632734 (37)	total: 3.89s	remaining: 47.3s
38:	learn: 0.6937035	test: 1.3611680	best: 1.3611680 (38)	total: 4s	remaining: 47.3s
39:	learn: 0.6838957	test: 1.3586775	best: 1.3586775 (39)	total: 4.09s	remaining: 47s
40:	learn: 0.6761731	test: 1.3552238	best: 1.3552238 (40)	total: 4.18

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed: 66.2min finished


0:	learn: 2.0432978	test: 2.1366941	best: 2.1366941 (0)	total: 108ms	remaining: 54s
1:	learn: 1.9327597	test: 2.0841912	best: 2.0841912 (1)	total: 215ms	remaining: 53.6s
2:	learn: 1.8396669	test: 2.0370120	best: 2.0370120 (2)	total: 334ms	remaining: 55.3s
3:	learn: 1.7615454	test: 1.9957559	best: 1.9957559 (3)	total: 446ms	remaining: 55.3s
4:	learn: 1.6928592	test: 1.9580814	best: 1.9580814 (4)	total: 563ms	remaining: 55.8s
5:	learn: 1.6312693	test: 1.9238254	best: 1.9238254 (5)	total: 670ms	remaining: 55.1s
6:	learn: 1.5727174	test: 1.8797915	best: 1.8797915 (6)	total: 778ms	remaining: 54.8s
7:	learn: 1.5261628	test: 1.8519220	best: 1.8519220 (7)	total: 891ms	remaining: 54.8s
8:	learn: 1.4820865	test: 1.8248236	best: 1.8248236 (8)	total: 1.01s	remaining: 55s
9:	learn: 1.4424017	test: 1.8016591	best: 1.8016591 (9)	total: 1.12s	remaining: 55s
10:	learn: 1.4023443	test: 1.7711469	best: 1.7711469 (10)	total: 1.23s	remaining: 54.8s
11:	learn: 1.3696037	test: 1.7505788	best: 1.7505788 (11)	

## Revisando los mejores parámetros

In [ ]:
print("Best GradientBoostingClassifier Params:", Grid_gbct.best_params_)

Best GradientBoostingClassifier Params: {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 5, 'loss': 'deviance', 'learning_rate': 0.2}


In [28]:
print("Best XGBoostClassifier Params:", Grid_xgb.best_params_)

Best XGBoostClassifier Params: {'n_estimators': 600, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 1}


In [115]:
print("Best LGBMClassifier Params:", Grid_lgbm.best_params_)

Best LGBMClassifier Params: {'objective': 'l', 'n_estimators': 600, 'max_depth': 7, 'learning_rate': 0.1}


In [40]:
print("Best CatboostClassifier Params:", Grid_cbc.best_params_)

Best CatboostClassifier Params: {'random_strength': 0.5, 'max_ctr_complexity': 3, 'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 500, 'depth': 10, 'border_count': 200, 'bagging_temperature': 0.09}


### Creando las predicciones

In [ ]:
print("Best GradientBoostingClassifier Params:", Grid_gbct.best_score_)

Best GradientBoostingClassifier Params: 0.6661951907503455


In [31]:
print("Best XGBoostClassifier Params:", Grid_xgb.best_score_)

Best XGBoostClassifier Params: 0.6072266779907455


In [122]:
print("Best LGBMClassifier Params:", Grid_lgbm.best_score_)

Best LGBMClassifier Params: 0.7134963299888067


In [41]:
print("Best CatboostClassifier Params:", Grid_cbc.best_score_)

Best CatboostClassifier Params: 0.6795550464432607


### Creando las predicciones

In [ ]:
GradientBoostingpredictions_test = Grid_gbct.predict(X_test_scaled)

In [30]:
XGBoostpredictions_test = Grid_xgb.predict(X_test_scaled)

In [ ]:
LGBMpredictions_test = Grid_cbc.predict(X_test_scaled)

In [46]:
Catboostpredictions_test = Grid_cbc.predict(X_test_scaled)

## Creando el archivo Submission

In [60]:

def archivoSubmision(predictions_tests):
    readId = pd.read_csv('/content/drive/MyDrive/test.csv')

    predictions_genetic_disorder_test = []
    predictions_disorder_subclass_test = []

    for myString in predictions_tests:
        genetic_disorder, disorder_subclass = myString[0].split('<->')
        predictions_genetic_disorder_test.append(genetic_disorder) 
        predictions_disorder_subclass_test.append(disorder_subclass)

    
    submission = pd.DataFrame({
            "Patient Id": readId["Patient Id"],
            "Genetic Disorder": predictions_genetic_disorder_test,
            "Disorder Subclass": predictions_disorder_subclass_test,
        })
    return submission

    def archivoSubmisionCatboost(predictions_tests):
    readId = pd.read_csv('/content/drive/MyDrive/test.csv')

    predictions_genetic_disorder_test = []
    predictions_disorder_subclass_test = []

    for myString in predictions_tests:
        genetic_disorder, disorder_subclass = myString[0].split('<->')
        predictions_genetic_disorder_test.append(genetic_disorder) 
        predictions_disorder_subclass_test.append(disorder_subclass)

    
    submission = pd.DataFrame({
            "Patient Id": readId["Patient Id"],
            "Genetic Disorder": predictions_genetic_disorder_test,
            "Disorder Subclass": predictions_disorder_subclass_test,
        })
    return submission





In [ ]:
archivoSubmision(GradientBoostingpredictions_test).to_csv('GradientBoostingsubmission.csv', index=False)

In [ ]:
archivoSubmision(XGBoostpredictions_test).to_csv('XGBoostsubmission.csv', index=False)

In [ ]:
archivoSubmision(LGBMpredictions_test).to_csv('LGBMsubmission.csv', index=False)

In [61]:
archivoSubmisionCatboost(Catboostpredictions_test).to_csv('Catboostsubmission.csv', index=False)

## Guardando los modelos

In [ ]:
with open('GradientBoostingmodel.model', "wb") as archivo_salida:
    pickle.dump(Grid_gbct.best_estimator_, archivo_salida)

In [38]:
with open('XGBoostmodel.model', "wb") as archivo_salida:
    pickle.dump(Grid_xgb.best_estimator_, archivo_salida)

In [ ]:
with open('LGBMmodel.model', "wb") as archivo_salida:
    pickle.dump(Grid_lgbm.best_estimator_, archivo_salida)    

In [62]:
with open('Catboostmodel.model', "wb") as archivo_salida:
    pickle.dump(Grid_cbc.best_estimator_, archivo_salida)